In [1]:
%matplotlib inline

In [2]:
import nibabel as nib
import numpy as np
import pandas as pd
import os.path as op

/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
import AFQ.registration as afr

In [4]:
from nilearn.input_data import NiftiMasker

In [5]:
func_filename = op.join(op.expanduser('~'), 'data', 'rtfmri', 'func_0000.nii.gz')

In [6]:
func_img = nib.load(func_filename)

In [7]:
func_data = func_img.get_data()

In [8]:
conditions = pd.read_csv(op.join(op.expanduser('~'), 'data', 'rtfmri', 'timingFiles', 'fullRunLabels.txt'), header=None)

In [9]:
condition_mask = conditions[0].isin(['Rest', 'Active'])
conditions = conditions[condition_mask]
func_data = func_data[..., condition_mask]

In [10]:
reference_data = func_img.dataobj[..., 0]

In [11]:
# Here we define our ML pipeline: feature selection using ANOVA followed by an SVM classifier:

# Define the dimension reduction to be used.
# Here we use a classical univariate feature selection based on F-test,
# namely Anova. When doing full-brain analysis, it is better to use
# SelectPercentile, keeping 5% of voxels
# (because it is independent of the resolution of the data).
from sklearn.feature_selection import SelectPercentile, f_classif
feature_selection = SelectPercentile(f_classif, percentile=5)

from sklearn.svm import SVC
svc = SVC(kernel='linear')

from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

# We'll use point by point accuracy score: 
from sklearn.metrics import accuracy_score

In [12]:
fit_data = func_data[..., 0, None]

In [13]:
accuracies = []

for idx in range(1, func_data.shape[-1]):
    print("Fit data shape {0}".format(fit_data.shape))
    
    print("Incoming data: ", idx)
    this_data = func_img.dataobj[..., idx]

    print("Register incoming data to the reference image")
    this_data, _ = afr.affine_registration(this_data,
                                           reference_data)
    print("Create full series for this iteration")
    fit_data = np.concatenate([fit_data, this_data[..., None]], axis=-1)
    
    if idx > 10:
        mask_data = np.zeros(this_data.shape)
        mask_data[this_data > 0] = 1
        masker = NiftiMasker(mask_img=nib.Nifti1Image(mask_data, func_img.affine), #smoothing_fwhm=4,
                             standardize=True, memory_level=1)

        X = masker.fit_transform(nib.Nifti1Image(fit_data, func_img.affine))
        anova_svc.fit(X[:idx], conditions[:idx])
        y_pred = anova_svc.predict(X[idx].reshape(1, -1))
        accuracies.append(accuracy_score(y_pred, conditions.iloc[idx]))

Fit data shape (64, 64, 30, 1)
Incoming data:  1
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration
Fit data shape (64, 64, 30, 2)
Incoming data:  2
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration
Fit data shape (64, 64, 30, 3)
Incoming data:  3
Register incoming data to the referen

/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [  3403   5125   5126  21796  23484  85097  86934  88752  90586  90685
  95925  99278 100896 100897 102477 102483 102516 104081 104098 107018
 107219 107245 108809 108812 108819 110359 110364 111925] are constant.
  UserWarning)
/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountere

Fit data shape (64, 64, 30, 12)
Incoming data:  12
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration


/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    58     70   1725  13502  13522  21793  30619  30636  30637  32405
  32406  34222  34223  37848  37849  37876  37877  39739  39740  81517
  81540  90720  95955  99324  99329  99331 100924 100932 100937 100941
 100947 102466 102523 102534 102572 104086 104090 104104 105686 105755
 107321 107322 108923 108933 108940 108942 108943] are constant.
  UserWarning)
/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/Users/arokem/.virtual

Fit data shape (64, 64, 30, 13)
Incoming data:  13
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration


...

Fit data shape (64, 64, 30, 127)
Incoming data:  127
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration


/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Fit data shape (64, 64, 30, 128)
Incoming data:  128
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration


/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/arokem/.virtualenvs/rtfmri/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


IndexError: single positional indexer is out-of-bounds